# Find tracks

Compare experimental bubble histories to bubble history correlations. Bubble detection and linking performed by Trackpy, an implementation of the Crocker-Grier algorithm [@allanTrackpy2018; @crockerMethodsDigitalVideo1996].


In [ ]:
from boilercv.docs.nbs import init

init()

import colorcet  # noqa: F401  # Registers "cet_" prefixed colormaps in plt.colormaps
import numpy as np
import seaborn as sns
from matplotlib import colormaps  # type: ignore  # pyright 1.1.333
from matplotlib.figure import Figure
from matplotlib.patches import Rectangle
from matplotlib.pyplot import subplot_mosaic, subplots
from pandas import CategoricalDtype, DataFrame, read_hdf

from boilercv.correlations import (
    dimensionless_bubble_diameter_florschuetz,
    dimensionless_bubble_diameter_yuan,
    fourier,
    jakob,
    kinematic_viscosity,
    prandtl,
    reynolds,
    thermal_diffusivity,
)
from boilercv.data.sets import get_dataset
from boilercv.docs.nbs import HIDE, display_dataframe_with_math, init, nowarn
from boilercv.experiments.e230920_subcool import (
    GBC,
    M_TO_MM,
    OBJECTS,
    THERMAL_DATA,
    Col,
    get_cat_colorbar,
    get_first_from_palette,
    plot_composite_da,
    transform_cols,
)
from boilercv.images import scale_bool

with nowarn(capture=True):
    import trackpy as tp

tp.quiet()

TIME = "2023-09-20T17:14:18"
"""Timestamp of the trial to be analyzed."""

# Plotting
FIGURES: list[Figure] = []
"""Notebook figures available for export."""
WARM_PALETTE = colormaps["cet_glasbey_warm"]
"""For plotting one approach."""
COOL_PALETTE = colormaps["cet_glasbey_cool"]
"""For plotting the other approach."""
PLOT_SUBCOOLING_TEXT = False
"""Whether to plot subcooling text on the final plot."""
PLOT_PRIOR_BOUNDS = False
"""Whether to plot the prior approach bounds on the final plot."""

# Track tuning
SEARCH_RANGE = 30
"""Pixel range to search for the next bubble."""
MEMORY = 5
"""Frames to remember a bubble."""
YPX_SURFACE_THRESHOLD = 400
"""Vertical position of bubble centroids considered attached to the surface."""
YPX_DEPARTURE_THRESHOLD = 420
"""Vertical position of bubble centroids considered to have departed the surface."""
MINIMUM_LIFETIME = 0.010  # (s)
"""Minimum bubble lifetime to consider."""

# Physical parameters
PX_PER_M = 20997.3753
LATENT_HEAT_OF_VAPORIZATION = 2.23e6  # J/kg
LIQUID_DENSITY = 960  # kg/m^3
LIQUID_DYNAMIC_VISCOSITY = 2.88e-4  # Pa-s
LIQUID_ISOBARIC_SPECIFIC_HEAT = 4213  # J/kg-K
LIQUID_THERMAL_CONDUCTIVITY = 0.676  # W/m-K
VAPOR_DENSITY = 0.804  # kg/m^3
TIME_CORR = np.logspace(-6, 0) / 2  # s
"""Time range for correlation plots."""

HIDE

In [ ]:
path_time = TIME.replace(":", "-")
video = scale_bool(get_dataset(path_time, stage="filled")["video"])
frametime = np.diff(video.time.values).mean()
objects: DataFrame = read_hdf((OBJECTS / f"objects_{path_time}").with_suffix(".h5"))  # type: ignore
subcooling = read_hdf(THERMAL_DATA).subcool[TIME]
minimum_frame_lifetime = int(MINIMUM_LIFETIME // frametime)

HIDE

## Find bubbles in each frame and link them

Detect individual bubbles in each frame, and then link detections across frames by application of the Crocker-Grier tracking algorithm, which takes into account centroid proximity and expected positions [@crockerMethodsDigitalVideo1996].

Initial and lifetime characteristics of long-lived bubbles are shown in **Table&NonBreakingSpace;1**. All bubbles departing the surface have an initial depth, $y$, close to the actual boiling surface, and a bimodal distribution in initial $x$, close to active nucleation sites. This information is used to determine surface and departure $y$ thresholds for alignment of bubble departures.


In [ ]:
tracks = (
    tp.link(f=objects, search_range=SEARCH_RANGE, memory=MEMORY)
    .rename(columns={"x": "xpx", "y": "ypx"})
    .assign(
        frame_lifetime=(
            lambda df: df.groupby("particle", **GBC)["frame"].transform("count")
        )
    )
    .sort_values(["frame_lifetime", "particle", "frame"], ascending=[False, True, True])
    .assign(
        bubble=(
            lambda df: df.groupby("particle", **GBC)
            .ngroup()
            .astype(CategoricalDtype(ordered=True))
        ),
        dypx=lambda df: df.groupby("bubble", **GBC)[["ypx"]].diff().fillna(0),
        dxpx=lambda df: df.groupby("bubble", **GBC)[["xpx"]].diff().fillna(0),
        diameter=lambda df: 4 * df["size"] / PX_PER_M,  # radius of gyration to diam
        y=lambda df: df["ypx"] / PX_PER_M,
        x=lambda df: df["xpx"] / PX_PER_M,
        dy=lambda df: df["dypx"] / PX_PER_M / frametime,
        dx=lambda df: df["dxpx"] / PX_PER_M / frametime,
        distance=lambda df: np.linalg.norm(df[["dx", "dy"]].abs(), axis=1),
        time=lambda df: video.sel(frame=df["frame"].values)["time"],
        lifetime=lambda df: df["frame_lifetime"] * frametime,
    )
    .drop(columns=["particle"])
)

display_dataframe_with_math(
    tracks.groupby("bubble", **GBC)
    .head(1)
    .set_index("bubble")
    .query(f"frame_lifetime > {minimum_frame_lifetime}")
    .head(16)
    .pipe(
        transform_cols,
        cols=[
            Col("lifetime", "Lifetime", "s"),
            Col("time", r"$t_0$", "s"),
            Col("diameter", r"$d_{b0}$", **M_TO_MM),
            Col("y", r"$y_{b0}$", **M_TO_MM),
            Col("x", r"$x_{b0}$", **M_TO_MM),
        ],
    )
)

**Table&NonBreakingSpace;1**: Selected properties of long-lived bubbles
Bubbles are identified by a unique particle number. Their lifetime, the time of their first appearance, their initial diameter and elevation, and lifetime histograms of selected characteristics are shown.


## History of long-lived bubbles

The paths taken by long-lived bubbles are shown in **Figure&NonBreakingSpace;2**. Two active nucleation sites are responsible for all bubbles produced, and bubbles departing from each nucleation site take one of a few predominant paths during the short period of observation.


In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
plot_composite_da(video, ax)
long_lived_objects = tracks.query(f"frame_lifetime > {minimum_frame_lifetime}")
palette, data = get_cat_colorbar(
    ax,
    palette=COOL_PALETTE,
    data=long_lived_objects.pipe(
        transform_cols,
        [
            hue := Col("bubble", "Individual bubble"),
            x := Col("xpx", "x", "px"),
            y := Col("ypx", "y", "px"),
        ],
    ),
    col=hue.new,
)
sns.scatterplot(
    ax=ax,
    edgecolor="none",
    s=10,
    x=x.new,
    y=y.new,
    hue=hue.new,
    legend=False,  # type: ignore  # pyright 1.1.333
    palette=palette,
    data=data,
)

if PLOT_PRIOR_BOUNDS:
    ax.add_patch(
        Rectangle(
            xy=(130, 240),
            width=(250 - 130),
            height=(460 - 240),
            edgecolor="blue",
            facecolor="none",
            linewidth=2,
        )
    )

HIDE

**Figure&NonBreakingSpace;1**: Long-lived bubble tracks  
Bubble tracks indicated by the positions of detected centroids over time.

## Aligning bubble departures

Exclude bubbles that did not originate from the boiling surface, or that had already departed the surface at the time of recording. Consider a bubble to have departed the surface when its centroid crosses a departure threshold which is about one average bubble diameter above the boiling surface. Define the origin for time of departure for each bubble in this fashion. The resulting time history in **Figure&NonBreakingSpace;2** shows bubble depth, velocity, and diameter for the remainder of its visible lifetime.

Most bubbles rise and collapse at similar rates. Two bubbles rise slower than the rest, but seem to collapse at about the same rate as others.


In [ ]:
departing_long_lived_objects = (
    # Find rows corresponding to stagnant or invalid bubbles
    long_lived_objects.groupby("bubble", **GBC)
    .apply(
        # Don't assign any other columns until invalid rows have been filtered out
        lambda df: df.assign(
            yinitpx=lambda df: df["ypx"].iat[0],
            # Initial y position is close to the surface
            began=lambda df: df["yinitpx"] > YPX_SURFACE_THRESHOLD,
            # When the bubble gets far enough away from the surface
            departed=lambda df: df["ypx"] < YPX_DEPARTURE_THRESHOLD,
        )
    )
    # Filter out invalid rows and drop the columns used to determine validity
    .pipe(lambda df: df[df["began"] & df["departed"]])
    .drop(columns=["began", "departed"])
    # Groupby again after filtering out invalid rows
    .groupby("bubble", **GBC)
    # Now columns that depend on the initial row (*.iat[0]) can be assigned
    .apply(
        lambda df: df.assign(
            frame=lambda df: df["frame"] - df["frame"].iat[0],
            time=lambda df: df["time"] - df["time"].iat[0],
            frame_lifetime=lambda df: df["frame"].iat[-1] - df["frame"].iat[0],
            lifetime=lambda df: df["frame_lifetime"] * frametime,
            yinit=lambda df: df["y"].iat[0],
            xinit=lambda df: df["x"].iat[0],
            init_diameter=lambda df: df["diameter"].iat[0],
            dyinit=lambda df: df["dy"].iat[0],
            dyinitpx=lambda df: df["dy"].iat[0],
            max_diameter=lambda df: df["diameter"].max(),
        )
    )
)
cols = [
    hue := Col("bubble", "Individual bubble"),
    x := Col("time", "Time after departure", "s"),
    y := Col("y", "Depth", **M_TO_MM),
    v := Col("dy", "Velocity", "m/s", "mm/s"),
    d := Col("diameter", "Diameter", **M_TO_MM),
]
figure, axs = subplot_mosaic([[y.new], [v.new], [d.new]])
FIGURES.append(figure)
figure.set_size_inches(6, 10)
for plot, ax in axs.items():
    palette, data = get_cat_colorbar(
        ax,
        hue.new,
        COOL_PALETTE,
        departing_long_lived_objects.pipe(transform_cols, cols),
    )
    sns.scatterplot(
        ax=ax,
        edgecolor="none",
        s=10,
        alpha=0.4,
        x=x.new,
        y=plot,
        hue=hue.new,
        legend=False,  # type: ignore  # pyright 1.1.333
        palette=palette,
        data=data,
    )

**Figure&NonBreakingSpace;2**: Time history of long-lived bubbles
Bubble depth, velocity, and diameter plotted over time.

Histograms of individual bubble statistics are shown in **Figure&NonBreakingSpace;3**. Because no bubbles completely collapse, bubble lifetimes correspond to the duration of time between their departure from the boiling surface and rising past the upper limit of the camera viewpoint. The maximum bubble diameter is about 3&NonBreakingSpace;mm.

The bimodal distribution of initial $x$ positions is also evident, corresponding to two active nucleation sites. Initial bubble velocity at departure tends to be about 250&NonBreakingSpace;mm/s.


In [ ]:
figure, ax = subplots()
FIGURES.append(figure)
(
    departing_long_lived_objects.pipe(
        transform_cols,
        [
            Col("bubble"),
            Col("lifetime", "Lifetime", "s"),
            Col("max_diameter", r"$d_{max}$", **M_TO_MM),
            Col("init_diameter", r"$d_{b0}$", **M_TO_MM),
            Col("yinit", r"$y_{b0}$", **M_TO_MM),
            Col("xinit", r"$x_{b0}$", **M_TO_MM),
            Col("dyinit", r"$v_{y0}$", old_unit="m/s", new_unit="mm/s", scale=1000),
        ],
    )
    .groupby("bubble", **GBC)
    .mean()
    .set_index("bubble")
    .hist(ax=ax)
)

HIDE

**Figure&NonBreakingSpace;3**: Histograms of individual bubble statistics  
Shows bubble lifetime, maximum diameter, and bubble properties at departure.


# Correlations

One correlation for bubble history of direct contact condensation of vapor bubbles in a subcooled liquid such considers a stagnant bubble in liquid dominated by heat transfer, which can be represented as

$$
\beta = 1 - 4{Ja}\sqrt\frac{{Fo}_0}{\pi}
$$

where $\beta$ is the dimensionless bubble diameter $D/D_0$ with $D_0$ being the initial bubble diameter, ${Ja}$ is the Jakob number $\rho_l c_{pl}\Delta T_{sub}/\rho_v h_{fg}$ , and ${Fo}_0$ is the Fourier number $\alpha t/D_{b0}^2$ [@tangReviewDirectContact2022; @florschuetzMechanicsVaporBubble1965]. This correlation was derived from analysis of the physical phenomena, and does not incorporate a fit to experimental data.

A later correlation, one which does incoprorate a fit to experimental data, is

$$
\beta = \left(1-1.8{Re}_{b0}^{1/2}Pr^{1/3}{Ja}{Fo}_0\left(1-\frac{{Ja}^{1/10}{Fo}_0}{2}\right)\right)^{2/3}
$$

where ${Re}_{b0}$ and ${Pr}$ are the bubble Reynolds and liquid Prandtl numbers, respectively [@tangReviewDirectContact2022; @yuandewenCondensationHeatTransfer2009]. Experimental bubble data is nondimensionalized by initial bubble diameter, and correlations are plotted against experimental data in **Figure&NonBreakingSpace;5**. Correlations are plotted for the average initial bubble diameter and velocity of the population of bubbles studied.

Bubble histories seem to correspond roughly with the analytical model by Florschuetz and Chao initially, with later times corresponding to the Yuan et al. model. The present bubble data shows about 0.5&NonBreakingSpace;K subcooling. Since correlations are sensitive to subcool temperature, this motivates the collection of bubble data over a wider range of subcooling.


In [ ]:
object_averages = (
    departing_long_lived_objects.set_index("bubble")
    .groupby("bubble", **GBC)
    .mean()
    .mean()
)
liquid_kinematic_viscosity = kinematic_viscosity(
    density=LIQUID_DENSITY, dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY
)
liquid_thermal_diffusivity = thermal_diffusivity(
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
    density=LIQUID_DENSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
)
bubble_initial_reynolds = reynolds(
    velocity=abs(object_averages["dyinit"]),
    characteristic_length=object_averages["init_diameter"],
    kinematic_viscosity=liquid_kinematic_viscosity,
)
liquid_prandtl = prandtl(
    dynamic_viscosity=LIQUID_DYNAMIC_VISCOSITY,
    isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    thermal_conductivity=LIQUID_THERMAL_CONDUCTIVITY,
)
bubble_jakob = jakob(
    liquid_density=LIQUID_DENSITY,
    vapor_density=VAPOR_DENSITY,
    liquid_isobaric_specific_heat=LIQUID_ISOBARIC_SPECIFIC_HEAT,
    subcooling=subcooling,
    latent_heat_of_vaporization=LATENT_HEAT_OF_VAPORIZATION,
)
bubble_fourier = fourier(
    liquid_thermal_diffusivity=liquid_thermal_diffusivity,
    initial_bubble_diameter=object_averages["init_diameter"],
    time=TIME_CORR,
)
nondimensionalized_departing_long_lived_objects = departing_long_lived_objects.assign(**{
    "Bubble Fourier number": lambda df: fourier(
        initial_bubble_diameter=df["init_diameter"],
        liquid_thermal_diffusivity=liquid_thermal_diffusivity,
        time=df["time"],
    ),
    "Dimensionless bubble diameter": (lambda df: df["diameter"] / df["init_diameter"]),
})
figure, ax = subplots()
FIGURES.append(figure)
ax.set_xlim(0, 0.04)
ax.set_ylim(0, 1.05)
sns.lineplot(
    ax=ax,
    data=(
        data := DataFrame(index=bubble_fourier).assign(  # type: ignore  # pyright 1.1.333
            **{
                "Florshuetz and Chao (1965)": dimensionless_bubble_diameter_florschuetz(
                    jakob=bubble_jakob, fourier=bubble_fourier
                ),
                "Yuan et al. (2009)": dimensionless_bubble_diameter_yuan(
                    bubble_initial_reynolds=bubble_initial_reynolds,
                    liquid_prandtl=liquid_prandtl,
                    bubble_jakob=bubble_jakob,
                    bubble_fourier=bubble_fourier,
                ),
            }
        )
    ),
    palette=get_first_from_palette(WARM_PALETTE, len(data.columns)).colors,  # type: ignore  # pyright 1.1.333
)
palette, data = get_cat_colorbar(
    ax,
    palette=COOL_PALETTE,
    data=nondimensionalized_departing_long_lived_objects.pipe(
        transform_cols,
        [
            hue := Col("bubble", "Individual bubble"),
            x := Col("Bubble Fourier number"),
            y := Col("Dimensionless bubble diameter"),
        ],
    ),
    col=hue.new,
)
sns.scatterplot(
    ax=ax,
    s=10,
    alpha=0.4,
    x=x.new,
    y=y.new,
    hue=hue.new,
    palette=palette,
    legend=False,  # type: ignore  # pyright 1.1.333
    data=data,
)
with nowarn():
    sns.move_legend(ax, "lower center", bbox_to_anchor=(0.5, 1), ncol=3)
if PLOT_PRIOR_BOUNDS:
    ax.add_patch(
        Rectangle(
            xy=(0.0002, 0.2),
            width=0.003,
            height=(1 - 0.2),
            edgecolor="blue",
            facecolor="none",
            linewidth=2,
        )
    )
if PLOT_SUBCOOLING_TEXT:
    ax.text(0.033, 0.92, f"{subcooling:.2f} K", fontsize=14)

HIDE

**Figure&NonBreakingSpace;5**: Comparison of bubble histories to correlations

Two correlations are shown. The early bubble history follows that of the analytical correlation by Florshuetz and Chao (1965), while the late bubble history follows that of Yuan et al (2009).
